# Central RL control notebook

In [1]:
import sys
from pymgrid import MicrogridGenerator as mg
import matplotlib.pyplot as plt
import numpy as np
import os
import time
from IPython.display import display,clear_output
import pandas as pd
from Storage import StorageSuite as st
from pathlib import Path

Time span constants

In [2]:
YEAR = 31579200
MONTH = 2570400
WEEK = 604800
DAY = 86400
HOUR = 3600
MINUTE = 60

### Generate Microgrids

In [3]:
n_grids = 2
env_list = []
for i in range(n_grids):
    env_list.append(st(r'C:\Users\thesu\Desktop\Research 2022 Microgrid\Research code repository\Micro-Grid-ML-algo\data\dummy_data_v1.csv',load=np.random.randint(1E5,6E5)))
env = mg.MicrogridGenerator(storage_suite_list=env_list)
env.generate_microgrid(verbose= False, interpolate= True)

{'li-ion': <Storage.Storage object at 0x0000020EFC4DE6B0>, 'flywheel': <Storage.Storage object at 0x0000020EFC4DE4D0>, 'flow': <Storage.Storage object at 0x0000020EF6788820>}
{'li-ion': <Storage.Storage object at 0x0000020EF6789390>, 'flywheel': <Storage.Storage object at 0x0000020EF67887C0>, 'flow': <Storage.Storage object at 0x0000020EF6789570>}


,load,load_size,cost_loss_load,cost_overgeneration,cost_co2,PV_rated_power,grid_weak,grid_power_import,grid_power_export,genset_polynom_order,genset_polynom_0,genset_polynom_1,genset_polynom_2,genset_rated_power,genset_pmin,genset_pmax,fuel_cost,genset_co2
0,349342,349342,10,1,0.1,216592.04,0,698684,698684,3,8.962852,0.699991,0.07143,388158,0.05,0.9,0.4,2


,load,load_size,cost_loss_load,cost_overgeneration,cost_co2,PV_rated_power,grid_weak,grid_power_import,grid_power_export,genset_polynom_order,genset_polynom_0,genset_polynom_1,genset_polynom_2,genset_rated_power,genset_pmin,genset_pmax,fuel_cost,genset_co2
0,377869,377869,10,1,0.1,294737.82,1,755738,755738,3,3.992596,0.578001,0.058802,419855,0.05,0.9,0.4,2


In [4]:
env.print_mg_parameters()

,load,load_size,cost_loss_load,cost_overgeneration,cost_co2,PV_rated_power,grid_weak,grid_power_import,grid_power_export,genset_polynom_order,genset_polynom_0,genset_polynom_1,genset_polynom_2,genset_rated_power,genset_pmin,genset_pmax,fuel_cost,genset_co2
0,349342,349342,10,1,0.1,216592.04,0,698684,698684,3,8.962852,0.699991,0.071430,388158,0.05,0.9,0.4,2
1,377869,377869,10,1,0.1,294737.82,1,755738,755738,3,3.992596,0.578001,0.058802,419855,0.05,0.9,0.4,2


In [5]:

mg0 = env.microgrids[0]
mg1 = env.microgrids[1]


In [6]:
li,flow,fly = env_list[0].unpack()
print(f'Litium: {round(li.soc*100,8)} %, {round(li.soc*li.max_energy)} kWh')
print(f'Flow: {round(flow.soc*100,8)} %, {round(flow.soc*flow.max_energy)} kWh')
print(f'Flywheel: {round(fly.soc*100,8)} %, {round(fly.soc*fly.max_energy)} kWh')

Litium: 100 %, 116447 kWh
Flow: 100 %, 116447 kWh
Flywheel: 100 %, 116447 kWh


In [10]:
pv = round(mg0.pv)
load = round(mg0.load)
control_dict = {            'pv_consummed': pv,
                            'li_charge': 0,
                            'li_discharge': 0,
                            'flow_charge': 0,
                            'flow_discharge': 0,
                            'flywheel_charge': 500000,
                            'flywheel_discharge': 0,
                            'grid_import': 0,
                            'grid_export':0,
                            'genset': 0
                           }
mg0.run(control_dict=control_dict)

Nothing Sent, nothing requested
Li-ion self-discharge = 0.8076512604566147, Li-ion charge = 0, Li-ion discharge 0
Nothing Sent, nothing requested
Flow self-discharge = 0.0, Flow charge = 0, Flow discharge 0
Flywheel self-discharge = 1244.9727740815856, Flywheel charge = 10169.92402924764, Flywheel discharge 0


{'load': 109501.9745066505,
 'hour': 1,
 'pv': 0.0,
 'grid_variables': 1.0,
 'grid_co2': 0.24088793096666666,
 'grid_price_import': 0.11,
 'grid_price_export': 0.0}

In [11]:
li,flow,fly = env_list[0].unpack()
print(f'Litium: {round(li.soc*100,3)} %, {round(li.soc*li.max_energy)} kWh')
print(f'Flow: {round(flow.soc*100,3)} %, {round(flow.soc*flow.max_energy)} kWh')
print(f'Flywheel: {round(fly.soc*100,3)} %, {round(fly.soc*fly.max_energy)} kWh')

Litium: 99.999 %, 116446 kWh
Flow: 100.0 %, 116447 kWh
Flywheel: 87.389 %, 101762 kWh


In [9]:
breaker()

NameError: name 'breaker' is not defined

In [ ]:
mg0._df_record_actual_production

In [ ]:
# mg0_status = mg0._grid_status_ts
# mg1_status = mg1._load_ts
# print(len(mg1_status))
# plt.plot(mg1_status[0:60])
# plt.show()
# plt.plot(mg0_status[0:DAY])
# plt.show()

In [ ]:
mg0.get_control_dict()

In [1]:
# Grid is up

    # PV is above load

    # PV is below load


# Grid is down

    # PV is above load

    # PV is below load
        


# Stock control dict

# control_dict = {            'pv_consummed': min(pv,load),
#                             'li_charge': 0,
#                             'li_discharge': 0,
#                             'flow_charge': 0,
#                             'flow_discharge': 0,
#                             'flywheel_charge': 0,
#                             'flywheel_discharge': 0,
#                             'grid_import': 0,
#                             'grid_export':0,
#                             'genset': 0
#                            }

def actions_agent(mg0, action):
    li_ion,flow,flywheel = mg0.ss.unpack()
    pv = round(mg0.pv)
    load = round(mg0.load)

    li_ion_charge_max = li_ion.power
    li_ion_discharge_max = li_ion.power
    li_ion_peak = li_ion.peak_discharge
    li_ion_soc = li_ion.soc
    li_ion_capa_to_charge = li_ion.capa_to_charge
    li_ion_capa_to_discharge = li_ion.capa_to_discharge

    flow_charge_max = flow.power
    flow_discharge_max = flow.power
    flow_peak = flow.peak_discharge
    flow_soc = flow.soc
    flow_capa_to_charge = flow.capa_to_charge
    flow_capa_to_discharge = flow.capa_to_discharge

    flywheel_charge_max = flywheel.power
    flywheel_discharge_max = flywheel.power
    flywheel_peak = flywheel.peak_discharge
    flywheel_soc = flywheel.soc
    flywheel_capa_to_charge = flywheel.capa_to_charge
    flywheel_capa_to_discharge = flywheel.capa_to_discharge
    
    soc_list = {'li_ion_soc': li_ion_soc, 'flow_soc': flow_soc, 'flywheel_soc': flywheel_soc}
    soc_list = sorted(soc_list, key=lambda x: (soc_list[x]))
    
    if action == 0:
        # NON-ISLANDED OVERGENERATION EXPORT
        control_dict = {    'pv_consummed': min(pv,abs(load)),
                            'li_charge': 0,
                            'li_discharge': 0,
                            'flow_charge': 0,
                            'flow_discharge': 0,
                            'flywheel_charge': 0,
                            'flywheel_discharge': 0,
                            'grid_import': 0,
                            'grid_export':pv-abs(load),
                            'genset': 0
                        }

########################################################################################################################
    if action == 1:
        # NON-ISLANDED OVERGENERATION EXCLUSIVE CHARGE AND EXPORT
        if soc_list[0] == 'li_ion_soc':
            # Charge li-ion exclusively
            control_dict = {    'pv_consummed': min(pv,abs(load)),
                                'li_charge': li_ion_charge_max,
                                'li_discharge': 0,
                                'flow_charge': 0,
                                'flow_discharge': 0,
                                'flywheel_charge': 0,
                                'flywheel_discharge': 0,
                                'grid_import': 0,
                                'grid_export': pv-abs(load)-min(pv-load,li_ion_charge_max),
                                'genset': 0
                            }

        elif soc_list[0] == 'flow_soc':
            # Charge flow exclusively
            control_dict = {    'pv_consummed': min(pv,abs(load)),
                                'li_charge': 0,
                                'li_discharge': 0,
                                'flow_charge': flow_charge_max,
                                'flow_discharge': 0,
                                'flywheel_charge': 0,
                                'flywheel_discharge': 0,
                                'grid_import': 0,
                                'grid_export': pv-abs(load)-min(pv-load,flow_charge_max),
                                'genset': 0
                            }

        elif soc_list[0] == 'flywheel_soc':
            # Charge flywheel exclusively
            control_dict = {    'pv_consummed': min(pv,abs(load)),
                                'li_charge': 0,
                                'li_discharge': 0,
                                'flow_charge': 0,
                                'flow_discharge': 0,
                                'flywheel_charge': flywheel_charge_max,
                                'flywheel_discharge': 0,
                                'grid_import': 0,
                                'grid_export': pv-abs(load)-min(pv-load,flywheel_charge_max),
                                'genset': 0
                            }

########################################################################################################################
    if action == 2:
        # ISLANDED OVERGENERATION EXCLUSIVE CHARGE
        if soc_list[0] == 'li_ion_soc':
            # Charge li-ion exclusively
            control_dict = {    'pv_consummed': min(pv,abs(load)),
                                'li_charge': min(pv-abs(load),li_ion_charge_max),
                                'li_discharge': 0,
                                'flow_charge': 0,
                                'flow_discharge': 0,
                                'flywheel_charge': 0,
                                'flywheel_discharge': 0,
                                'grid_import': 0,
                                'grid_export': 0,
                                'genset': 0
                            }

        elif soc_list[0] == 'flow_soc':
            # Charge flow exclusively
            control_dict = {    'pv_consummed': min(pv,abs(load)),
                                'li_charge': 0,
                                'li_discharge': 0,
                                'flow_charge': min(pv-abs(load),flow_charge_max),
                                'flow_discharge': 0,
                                'flywheel_charge': 0,
                                'flywheel_discharge': 0,
                                'grid_import': 0,
                                'grid_export': 0,
                                'genset': 0
                            }

        elif soc_list[0] == 'flywheel_soc':
            # Charge flywheel exclusively
            control_dict = {    'pv_consummed': min(pv,abs(load)),
                                'li_charge': 0,
                                'li_discharge': 0,
                                'flow_charge': 0,
                                'flow_discharge': 0,
                                'flywheel_charge': min(pv-abs(load),flywheel_charge_max),
                                'flywheel_discharge': 0,
                                'grid_import': 0,
                                'grid_export': 0,
                                'genset': 0
                            }
########################################################################################################################
    elif action == 3:
        # NON-ISLANDED OVERGENERATION COMBINED CHARGE NO EXPORT
        control_dict = {    'pv_consummed': min(pv,abs(load)),
                            'li_charge': min((pv-abs(load))/3,li_ion_charge_max),
                            'li_discharge': 0,
                            'flow_charge': min((pv-abs(load))/3,flow_charge_max),
                            'flow_discharge': 0,
                            'flywheel_charge': min((pv-abs(load))/3,flywheel_charge_max),
                            'flywheel_discharge': 0,
                            'grid_import': 0,
                            'grid_export': max(0, (pv-abs(load) - li_ion_charge_max - flow_charge_max - flywheel_charge_max)),
                            'genset': 0
                           }

########################################################################################################################
    elif action == 4:
        # ISLANDED OVERGENERATION COMBINED CHARGE
        control_dict = {    'pv_consummed': min(pv,abs(load)),
                            'li_charge': min((pv-abs(load))/3,li_ion_charge_max),
                            'li_discharge': 0,
                            'flow_charge': min((pv-abs(load))/3,flow_charge_max),
                            'flow_discharge': 0,
                            'flywheel_charge': min((pv-abs(load))/3,flywheel_charge_max),
                            'flywheel_discharge': 0,
                            'grid_import': 0,
                            'grid_export':0,
                            'genset': 0
                           }

########################################################################################################################
    elif action == 5:
        # NON-ISLANDED OVERGENERATION CONDITIONAL CHARGE
        control_dict = {        'pv_consummed': min(pv,abs(load)),
                                'li_charge': li_ion_charge_max,
                                'li_discharge': 0,
                                'flow_charge': 0,
                                'flow_discharge': 0,
                                'flywheel_charge': 0,
                                'flywheel_discharge': 0,
                                'grid_import': 0,
                                'grid_export': pv-abs(load)-min(pv-load,li_ion_charge_max),
                                'genset': 0
                            }

        if li_ion_soc == 1:
            control_dict = {    'pv_consummed': min(pv,abs(load)),
                                'li_charge': 0,
                                'li_discharge': 0,
                                'flow_charge': flow_charge_max,
                                'flow_discharge': 0,
                                'flywheel_charge': 0,
                                'flywheel_discharge': 0,
                                'grid_import': 0,
                                'grid_export': pv-abs(load)-min(pv-load,flow_charge_max),
                                'genset': 0
                            }
        elif (li_ion_soc, flow_soc) == (1,1):
            control_dict = {    'pv_consummed': min(pv,abs(load)),
                                'li_charge': 0,
                                'li_discharge': 0,
                                'flow_charge': 0,
                                'flow_discharge': 0,
                                'flywheel_charge': flywheel_charge_max,
                                'flywheel_discharge': 0,
                                'grid_import': 0,
                                'grid_export': pv-abs(load)-min(pv-load,flywheel_charge_max),
                                'genset': 0
                            }

########################################################################################################################
    elif action == 6:
        # ISLANDED OVERGENERATION CONDITIONAL CHARGE
        control_dict = {    'pv_consummed': min(pv,abs(load)),
                                'li_charge': li_ion_charge_max,
                                'li_discharge': 0,
                                'flow_charge': 0,
                                'flow_discharge': 0,
                                'flywheel_charge': 0,
                                'flywheel_discharge': 0,
                                'grid_import': 0,
                                'grid_export': 0,
                                'genset': 0
                            }

        if li_ion_soc == 1:
            control_dict = {    'pv_consummed': min(pv,abs(load)),
                                'li_charge': 0,
                                'li_discharge': 0,
                                'flow_charge': flow_charge_max,
                                'flow_discharge': 0,
                                'flywheel_charge': 0,
                                'flywheel_discharge': 0,
                                'grid_import': 0,
                                'grid_export': 0,
                                'genset': 0
                            }
        elif (li_ion_soc, flow_soc) == (1,1):
            control_dict = {    'pv_consummed': min(pv,abs(load)),
                                'li_charge': 0,
                                'li_discharge': 0,
                                'flow_charge': 0,
                                'flow_discharge': 0,
                                'flywheel_charge': flywheel_charge_max,
                                'flywheel_discharge': 0,
                                'grid_import': 0,
                                'grid_export': 0,
                                'genset': 0
                            }

########################################################################################################################
    if action == 7:
        # NON-ISLANDED UNDERGENERATION EXCLUSIVE CHARGE
        if soc_list[0] == 'li_ion_soc':
            # Charge li-ion exclusively
            control_dict = {    'pv_consummed': min(pv,abs(load)),
                                'li_charge': min(abs(abs(load)-pv),li_ion_charge_max),
                                'li_discharge': 0,
                                'flow_charge': 0,
                                'flow_discharge': 0,
                                'flywheel_charge': 0,
                                'flywheel_discharge': 0,
                                'grid_import': 0,
                                'grid_export': pv-abs(load)-min(pv-load,li_ion_charge_max),
                                'genset': 0
                            }

        elif soc_list[0] == 'flow_soc':
            # Charge flow exclusively
            control_dict = {    'pv_consummed': min(pv,abs(load)),
                                'li_charge': 0,
                                'li_discharge': 0,
                                'flow_charge': min(abs(abs(load)-pv),flow_charge_max),
                                'flow_discharge': 0,
                                'flywheel_charge': 0,
                                'flywheel_discharge': 0,
                                'grid_import': 0,
                                'grid_export': pv-abs(load)-min(pv-load,flow_charge_max),
                                'genset': 0
                            }

        elif soc_list[0] == 'flywheel_soc':
            # Charge flywheel exclusively
            control_dict = {    'pv_consummed': min(pv,abs(load)),
                                'li_charge': 0,
                                'li_discharge': 0,
                                'flow_charge': 0,
                                'flow_discharge': 0,
                                'flywheel_charge': min(abs(abs(load)-pv),flywheel_charge_max),
                                'flywheel_discharge': 0,
                                'grid_import': 0,
                                'grid_export': 0,
                                'genset': 0
                            }

########################################################################################################################
    if action == 8:
        # ISLANDED UNDERGENERATION EXCLUSIVE DISCHARGE
        if soc_list[2] == 'li_ion_soc': # Check if Li ion battery is the highest charged storage device
            # Charge li-ion exclusively
            control_dict = {    'pv_consummed': min(pv,abs(load)),
                                'li_charge': 0,
                                'li_discharge': min(abs(abs(load)-pv),li_ion_discharge_max),
                                'flow_charge': 0,
                                'flow_discharge': 0,
                                'flywheel_charge': 0,
                                'flywheel_discharge': 0,
                                'grid_import': 0,
                                'grid_export': 0,
                                'genset': 0
                            }
            if li_ion_discharge_max < abs(abs(load)-pv): # Use Genset
                control_dict = {'pv_consummed': min(pv,abs(load)),
                                'li_charge': 0,
                                'li_discharge': min(abs(abs(load)-pv),li_ion_discharge_max),
                                'flow_charge': 0,
                                'flow_discharge': 0,
                                'flywheel_charge': 0,
                                'flywheel_discharge': 0,
                                'grid_import': 0,
                                'grid_export': 0,
                                'genset': 1
                                }

        elif soc_list[2] == 'flow_soc': # Check if flow battery is the highest charged storage device
            # Charge flow exclusively
            control_dict = {    'pv_consummed': min(pv,abs(load)),
                                'li_charge': 0,
                                'li_discharge': 0,
                                'flow_charge': 0,
                                'flow_discharge': min(abs(abs(load)-pv),flow_discharge_max),
                                'flywheel_charge': 0,
                                'flywheel_discharge': 0,
                                'grid_import': 0,
                                'grid_export': 0,
                                'genset': 0
                            }
            if flow_discharge_max < abs(abs(load)-pv): # Use Genset
                control_dict = {'pv_consummed': min(pv,abs(load)),
                                'li_charge': 0,
                                'li_discharge': 0,
                                'flow_charge': 0,
                                'flow_discharge': min(abs(abs(load)-pv),flow_discharge_max),
                                'flywheel_charge': 0,
                                'flywheel_discharge': 0,
                                'grid_import': 0,
                                'grid_export': 0,
                                'genset': 1
                                }

        elif soc_list[2] == 'flywheel_soc': # Check if flywheel is the highest charged storage device
            # Charge flywheel exclusively
            control_dict = {    'pv_consummed': min(pv,abs(load)),
                                'li_charge': 0,
                                'li_discharge': 0,
                                'flow_charge': 0,
                                'flow_discharge': 0,
                                'flywheel_charge': min(abs(abs(load)-pv),flywheel_charge_max),
                                'flywheel_discharge': 0,
                                'grid_import': 0,
                                'grid_export': 0,
                                'genset': 0
                            }
            if flow_discharge_max < abs(abs(load)-pv): # Use Genset
                control_dict = {'pv_consummed': min(pv,abs(load)),
                                'li_charge': 0,
                                'li_discharge': 0,
                                'flow_charge': 0,
                                'flow_discharge': 0,
                                'flywheel_charge': 0,
                                'flywheel_discharge': min(abs(abs(load)-pv),flow_discharge_max),
                                'grid_import': 0,
                                'grid_export': 0,
                                'genset': 1
                                }
        

    return control_dict

SyntaxError: '{' was never closed (1996093351.py, line 121)

In [ ]:
def change_name_action(idx):
# Grid is up

    # PV is above load
        # Battery full
            # action 0: grid up, export, battery full, pv > load
        # Battery empty
            # action 1: grid up, charge w/ genset, battery empty, pv > load
        # Battery not at full charge
            # action 2: grid up, charge w/ pv, battery < full, pv > load

    # PV is below load
        # Battery full
            # action 3: grid up, discharge, battery full, pv < load
        # Battery empty
            # action 4: grid up, import, battery empty, pv < load
                # import + charge until battery > 15%
        # Battery not at full charge
            # action 5: grid up, discharge, battery < full charge, pv < load
                # discharge if battery > 75 %
                # charge if battery < 15%

# Grid is down

    # PV is above load
        # Battery full
            # action 6: grid down, use pv, battery full, pv > load
        # Battery empty
            # action 7: grid down, charge w/ pv, battery empty, pv > load
        # Battery not at full charge
            # action 8: grid down, charge w/ pv, battery < full, pv > load

    # PV is below load
        # Battery full
            # action 9: grid down, discharge, battery full, pv > load
        # Battery empty
            # action 10: grid down, genset, battery empty, pv > load
        # Battery not at full charge
            # action 11: grid down, discharge, battery < full, pv > load

    if idx == 0:
        action_name = "grid up, export, battery full, pv > load"
    elif idx == 1:
        action_name = "grid up, charge w/ genset, battery empty, pv > load"
    elif idx == 2:
        action_name = "grid up, charge w/ pv, battery < full, pv > load"
    elif idx == 3:
        action_name = "grid up, discharge, battery full, pv < load"
    elif idx == 4:
        action_name = "grid up, import, battery empty, pv < load"
    elif idx == 5:
        action_name = "grid up, discharge, battery < full charge, pv < load"
    elif idx == 6 :
        action_name = "grid down, use pv, battery full, pv > load"
    elif idx == 7 :
        action_name = "grid down, charge w/ pv, battery empty, pv > load"
    elif idx == 8 :
        action_name = "grid down, charge w/ pv, battery < full, pv > load"
    elif idx == 9 :
        action_name = "grid down, discharge, battery full, pv > load"
    elif idx == 10 :
        action_name = "grid down, genset, battery empty, pv > load"
    elif idx == 11 :
        action_name = "grid down, discharge, battery < full, pv > load"
    return action_name

In [ ]:
def init_qtable(mg0, nb_action):
    cover_rate = (mg0._pv_ts*100)/mg0._load_ts
    state = []
    Q = {}
    cover_rate_range = np.arange(int(cover_rate.min()),int(cover_rate.max()))
    soc_range = np.arange(round(mg0.battery.soc_min,1),round(mg0.battery.soc_max+0.1,1),0.1)
    for i in cover_rate_range:
        for j in soc_range:
            j = round(j,1)
            state.append((i,j))
    
    # Initialize Q(s,a) at zero
        for s in state:
            Q[s] = {}
            for a in range(nb_action):
                Q[s][a] = 0
    return Q

I define a function for my exploration strategy. Here I've selected the epsilon greedy decreasing strategy

In [ ]:
def espilon_decreasing_greedy(action, epsilon, nb_action):
    
    p = np.random.random()

    if p < (1 - epsilon):
        randomm=0
        return action, randomm

    else: 
        randomm=1
        return np.random.choice(nb_action), randomm

In [ ]:
def max_dict(d):

    max_key = None
    max_val = float('-inf')


    for k,v in d.items():

        if v > max_val:

            max_val = v
            max_key = k

    return max_key, max_val

I also need to define a function to decrease and update my espilon 

In [ ]:
def update_epsilon(epsilon):
    
    epsilon = epsilon - epsilon *0.02
    
    if epsilon < 0.1:
        
        epsilon = 0.1
    
    return epsilon

In [ ]:
def print_welcome(idx):
    
    if idx == 0:
        print("------------------------------------")
        print("|        WELCOME TO PYMGRID        |")
        print("------------------------------------")
    elif idx == 1:
        
        print("t -     STATE  -  ACTION - COST")
        print("================================")
    

Now I define my agent function

In [ ]:
def training_Q_Learning(mg,horizon):
    
    nb_action = 12
    Q = init_qtable(mg,nb_action)
    nb_state = len(Q)
    nb_episode = 1000
    alpha = 0.1
    epsilon = 0.99
    gamma = 0.99
    
    record_cost = []
    t0 = time.time()
    t = t0
    print_training = "Training Progressing .   "
    print_welcome(0)
    print("\n")
    for e in range(nb_episode+1):
        
        if e == 0:
            value_print="\rEpisode " + str(e) +"/" + str(nb_episode)
            sys.stdout.write(value_print)
            sys.stdout.flush()
        
        else:
            now = time.time()
            if e == 1 or e == 50 or e == 100 or e == 200 or e == 300 or e == 400  or e == 450  or e == 485 or e == nb_episode-5:
                diff_time_t = now - t 
                total_time = (nb_episode+1) * diff_time_t
            
            now = time.time()
            
            diff_time = now - t0
            time_remaining = total_time - diff_time
            
            t = time.time()
        
        if e % 10 == 0:
        
            if print_training == "Training Progressing .   ":
                
                print_training = "Training Progressing ..  "
                
            elif print_training == "Training Progressing ..  ":
            
                print_training = "Training Progressing ... "
            
            else:
                
                print_training = "Training Progressing .   "

        value_print="\r"+ print_training +"Episode " + str(e) +"/" + str(nb_episode) 
        sys.stdout.write(value_print)
        sys.stdout.flush()
            
        episode_cost = 0
        mg.reset()
        
        cover_rate = int((mg.pv*100)/mg.load)
        soc = round(mg.battery.soc,1)
        s = (cover_rate, soc)
        
        a = max_dict(Q[s])[0]
        a,randomm = espilon_decreasing_greedy(a, epsilon, nb_action)
        for i in range (horizon):
            action = actions_agent(mg,a)

            action_name = change_name_action(a)
            status = mg.run(action)
            r = -mg.get_cost() 
            episode_cost += mg.get_cost()/3600
            cover_rate = int((mg.pv*100)/mg.load)
            soc = round(mg.battery.soc,1)
            s_ = (cover_rate, soc)
            a_ = max_dict(Q[s_])[0]
            if i == horizon-1:
                Q[s][a] += alpha*(r - Q[s][a])
            else:
             
                old_Q = Q[s][a]
                target = r + gamma*Q[s_][a_]
                td_error = target - Q[s][a]
                Q[s][a] = (1-alpha) * Q[s][a] + alpha * td_error
                
            s, a = s_, a_
       
        epsilon = update_epsilon(epsilon)
        
    return Q

In [ ]:
def testing_Q_Learning(mg, Q,horizon):
    cost_record = np.array((1,))
    mg.reset()
    cover_rate = int((mg.pv*100)/mg.load)
    soc = round(mg.battery.soc,1)
    s = (cover_rate, soc)
    a = max_dict(Q[s])[0]
    total_cost = 0
    print_welcome(1)
    export_counter = 0
    for i in range (1,horizon):

        action_name = change_name_action(a)
        action = actions_agent(mg, a)
        status = mg.run(action)
        # display(status) 
        cost = mg.get_cost()/3600
        cost_record = np.append(cost_record, cost+cost_record[i-1])
        total_cost += cost
        # if a == 0 or 2:
        #     export_counter += 1
        #     print(mg.get_cost()/3600)
        if i < 10:
            print(i," -",(int(cover_rate),soc),action_name, round(total_cost,2), "$")
        else:
            print(i,"-",(int(cover_rate),soc),action_name, round(total_cost,2), "$")
        cover_rate = int((mg.pv*100)/mg.load)
        soc = round(mg.battery.soc,1)
        s_ = (cover_rate, soc)
        a_ = max_dict(Q[s_])[0]

        s, a = s_, a_
    return cost_record,export_counter

In [ ]:
Q1 = training_Q_Learning(mg0,60)

In [ ]:
cost_record,export_counter = testing_Q_Learning(mg1,Q1, DAY)

In [ ]:
print(mg0.grid.power_import)

In [ ]:
print(sum(mg0._load_ts[0:DAY]))
arr = np.array(mg0._load_ts[0:DAY])
print(sum(arr)/3600)
